### temporary testing script

In [1]:
from langchain_google_genai import ChatGoogleGenerativeAI

import os
from dotenv import load_dotenv
load_dotenv() 

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash-lite",
    google_api_key=os.environ["GOOGLE_API_KEY"],
    temperature=0
)

g:\DataScience_projects\Text2Sql_chatbot\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Connect with sql db to ask some basic questions 

In [2]:
from langchain_experimental.sql import SQLDatabaseChain

from langchain.utilities import SQLDatabase
import cryptography
import urllib.parse

In [12]:
import cryptography
print(cryptography.__version__)

46.0.2


In [ ]:
db_user = "root"
db_password = urllib.parse.quote_plus("put_your_password")  # encodes special chars, here the problem was with '@' in password so that I have to apply urllib.parse.quote_plus
db_host = "localhost"
db_name = "zomato"

db = SQLDatabase.from_uri(f"mysql+mysqlconnector://{db_user}:{db_password}@{db_host}/{db_name}",sample_rows_in_table_info=3)


print(db.table_info)


CREATE TABLE `delhi ncr` (
	`NAME` TEXT, 
	`PRICE` INTEGER, 
	`CUSINE_CATEGORY` TEXT, 
	`CITY` TEXT, 
	`REGION` TEXT, 
	`URL` TEXT, 
	`PAGE NO` INTEGER, 
	`CUSINE TYPE` TEXT, 
	`TIMING` TEXT, 
	`RATING_TYPE` TEXT, 
	`RATING` TEXT, 
	`VOTES` TEXT
)DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB COLLATE utf8mb4_0900_ai_ci

/*
3 rows from delhi ncr table:
NAME	PRICE	CUSINE_CATEGORY	CITY	REGION	URL	PAGE NO	CUSINE TYPE	TIMING	RATING_TYPE	RATING	VOTES
POMP - Pizza On My Plate	800	Pizza,Italian,Desserts,Burger,American	Delhi NCR	Greater Kailash 1 (GK1)	https://www.zomato.com/ncr/pomp-pizza-on-my-plate-greater-kailash-gk-1-new-delhi/info	1	none	11am to 4am(Mon-Sun)	Excellent	4.6	1244
Shawarma House	500	Healthy Food,Wraps,Lebanese	Delhi NCR	Central Plaza Mall-- Golf Course Road	https://www.zomato.com/ncr/shawarma-house-golf-course-road-gurgaon/info	1	Quick Bites	11am to 4am(Mon-Sun)	Very Good	4.4	1408
Qwinny's	1000	Asian,Thai,Chinese,Sushi,PokÃ©	Delhi NCR	DLF Phase 4	https://www.zomato.com/ncr/qwinnys-d

#### Solution

In [9]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# LLM
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash-lite",
    google_api_key=os.environ["GOOGLE_API_KEY"],
    temperature=0,
)

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a SQL expert. Write a MySQL-compatible query for the user question."),
    ("human", "{question}")
])


# Format prompt to a string
user_question = "How many rows are in all the tables?"
formatted_prompt = prompt.format(question=user_question)

# Invoke LLM directly with string
response = llm.invoke(formatted_prompt)

print(response)


content='```sql\nSELECT SUM(TABLE_ROWS) AS total_rows\nFROM INFORMATION_SCHEMA.TABLES\nWHERE TABLE_SCHEMA = DATABASE();\n```' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash-lite', 'safety_ratings': []} id='run--d595da6e-28b8-4a8c-84c3-9fc010e51251-0' usage_metadata={'input_tokens': 32, 'output_tokens': 32, 'total_tokens': 64, 'input_token_details': {'cache_read': 0}}


In [10]:
# response is an AIMessage object
sql_text = response.content  # extract string from AIMessage

# Clean the SQL query by removing markdown formatting if present
cleaned_query = sql_text.strip('```sql\n').strip('\n```')

print(cleaned_query)


SELECT SUM(TABLE_ROWS) AS total_rows
FROM INFORMATION_SCHEMA.TABLES
WHERE TABLE_SCHEMA = DATABASE();


In [11]:
# Execute the cleaned query
result = db.run(cleaned_query)
print(result)

[(Decimal('58468'),)]
